In [1]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from scipy.io import arff
import pandas as pd
import numpy as np

In [2]:
# here using neural network without feature selecting the dataset to try match outcome prediction
data = arff.loadarff('FootPred_noMiss_numified_instancePruned.arff')
df = pd.DataFrame(data[0])
df.head()

home_player1_X_coord  home_player1_Y_coord  home_gk_diving  \
0                   1.0                   1.0            75.0   
1                   1.0                   1.0            83.0   
2                   1.0                   1.0            74.0   
3                   1.0                   1.0            77.0   
4                   1.0                   1.0            77.0   

   home_gk_handling  home_gk_kicking  home_gk_positioning  home_gk_reflexes  \
0              72.0             70.0                 74.0              77.0   
1              81.0             72.0                 80.0              83.0   
2              68.0             73.0                 64.0              76.0   
3              81.0             74.0                 80.0              79.0   
4              80.0             73.0                 78.0              84.0   

   home_player2_X_coord  home_player2_Y_coord  home_player2_crossing  ...  \
0                   2.0                   3.0                   73.0  ...   
1                   2.0                   3.0                   64.0  ...   
2                   2.0                   3.0                   74.0  ...   
3                   2.0                   3.0                   74.0  ...   
4                   2.0                   3.0                   70.0  ...   

   away_team_chanceCreationShootingClass  \
0                                    1.0   
1                                    3.0   
2                                    1.0   
3                                    1.0   
4                                    1.0   

   away_team_chanceCreationPositioningClass  away_team_defencePressure  \
0                                       1.0                       36.0   
1                                       1.0                       50.0   
2                                       1.0                       41.0   
3                                       1.0                       58.0   
4                                       1.0                       47.0   

   away_team_defencePressureClass  away_team_defenceAggression  \
0                             1.0                         45.0   
1                             1.0                         65.0   
2                             1.0                         39.0   
3                             1.0                         60.0   
4                             1.0                         45.0   

   away_team_defenceAggressionClass  away_team_defenceTeamWidth  \
0                               1.0                        49.0   
1                               1.0                        44.0   
2                               1.0                        51.0   
3                               1.0                        63.0   
4                               1.0                        49.0   

   away_team_defenceTeamWidthClass  away_team_defenceDefenderLineClass  class  
0                              1.0                                 1.0   b'0'  
1                              1.0                                 1.0   b'0'  
2                              1.0                                 1.0   b'0'  
3                              1.0                                 1.0   b'0'  
4                              1.0                                 1.0   b'0'  

[5 rows x 697 columns]

In [3]:
X = df.iloc[:, 0:695].astype(int)
y = df.iloc[:, 696].astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
oneC=np.arange(start=0, stop=347, step=1)
twoC=np.arange(start=348, stop=695, step=1)
X_train1=X_train.iloc[:,oneC]
X_train2=X_train.iloc[:,twoC]
X_test1=X_test.iloc[:,oneC]
X_test2=X_test.iloc[:,twoC]

#X_train1=tf.keras.utils.to_categorical(X_train1,347)
#X_train2=tf.keras.utils.to_categorical(X_train2,347)
#X_test1=tf.keras.utils.to_categorical(X_test1,347)
#X_test2=tf.keras.utils.to_categorical(X_test2,347)
#y_train=y_train.values.reshape(-1, 1)
#y_test=y_test.values.reshape(-1, 1)

#these are to mitigate errors for Sparse loss, no sparse comment em
#y_train=tf.keras.utils.to_categorical(y_train,3)
#y_test=tf.keras.utils.to_categorical(y_test,3)

In [4]:
#X_train1_normalized = preprocessing.normalize(X_train1, norm='max')
#X_train2_normalized = preprocessing.normalize(X_train2, norm='max')
#X_test1_normalized = preprocessing.normalize(X_test1, norm='max')
#X_test2_normalized = preprocessing.normalize(X_test2, norm='max')

In [5]:
input1=tf.keras.layers.Input(shape=(347,))
input2=tf.keras.layers.Input(shape=(347,))#shape=(347,)
merged=tf.keras.layers.Concatenate(axis=1)([input1, input2])
dropout1=tf.keras.layers.Dropout(0.8)(merged)
dense1=tf.keras.layers.Dense(16, activation='relu')(dropout1)
dropout2=tf.keras.layers.Dropout(0.8)(dense1)
dense2=tf.keras.layers.Dense(32, activation='relu')(dropout2)
dropout3=tf.keras.layers.Dropout(0.8)(dense2)
dense3=tf.keras.layers.Dense(64, activation='relu')(dropout3)
dropout4=tf.keras.layers.Dropout(0.8)(dense3)
dense4=tf.keras.layers.Dense(128, activation='relu')(dropout4)
dropout5=tf.keras.layers.Dropout(0.8)(dense4)
dense5=tf.keras.layers.Dense(256, activation='relu')(dropout5)
dropout6=tf.keras.layers.Dropout(0.8)(dense5)
dense6=tf.keras.layers.Dense(256, activation='relu')(dropout6)
dense7=tf.keras.layers.Dense(512, activation='relu')(dense6)
dense8=tf.keras.layers.Dense(1024, activation='relu')(dense7)
dense9=tf.keras.layers.Dense(2048, activation='relu')(dense8)
#tf.keras.layers.Flatten(),
output=tf.keras.layers.Dense(3, activation='softmax')(dense9)
model = tf.keras.models.Model([input1, input2], output)
#81,243,729

In [6]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy() #CategoricalCrossentropy()
opt = tf.keras.optimizers.Adam() ##adadelta showed some promise, well almost, learning_rate=0.01

In [7]:
model.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['accuracy'])

In [8]:
X_train1.shape

(7862, 347)

In [9]:
X_train1 = tf.convert_to_tensor(X_train1)
X_train2 = tf.convert_to_tensor(X_train2)
X_test1 = tf.convert_to_tensor(X_test1)
X_test2 = tf.convert_to_tensor(X_test2)
y_train = tf.convert_to_tensor(y_train)
model.fit([X_train1,X_train2], y_train, epochs=100,batch_size=1)

Epoch 1/100
7862/7862 [==============================] - 126s 16ms/step - loss: 5.5232 - accuracy: 0.3298
Epoch 2/100
7862/7862 [==============================] - 123s 16ms/step - loss: 1.2008 - accuracy: 0.3418
Epoch 3/100
7862/7862 [==============================] - 120s 15ms/step - loss: 1.1114 - accuracy: 0.3308
Epoch 4/100
7862/7862 [==============================] - 121s 15ms/step - loss: 1.1004 - accuracy: 0.3476
Epoch 5/100
7862/7862 [==============================] - 126s 16ms/step - loss: 1.1001 - accuracy: 0.3305
Epoch 6/100
7862/7862 [==============================] - 123s 16ms/step - loss: 1.0996 - accuracy: 0.3346
Epoch 7/100
7862/7862 [==============================] - 123s 16ms/step - loss: 1.0995 - accuracy: 0.3344
Epoch 8/100
7862/7862 [==============================] - 123s 16ms/step - loss: 1.1031 - accuracy: 0.3179
Epoch 9/100
7862/7862 [==============================] - 123s 16ms/step - loss: 1.1039 - accuracy: 0.3353
Epoch 10/100
7862/7862 [======================

KeyboardInterrupt: 

In [ ]:
model.evaluate([X_test1,X_test2],y_test)

In [ ]:
"""
xone= df.iloc[:, 0:327].astype(int)
xtwo= df.iloc[:, 328].astype(str)
xthree= df.iloc[:, 329].astype(int)
xfour= df.iloc[:, 330].astype(str)
xfive= df.iloc[:, 331].astype(int)
xsix= df.iloc[:, 332].astype(str)
xseven= df.iloc[:, 333].astype(str)
xeight= df.iloc[:, 334].astype(int)
xnine= df.iloc[:, 335].astype(str)
xten= df.iloc[:, 336].astype(int)
xeleven= df.iloc[:, 337].astype(str)
xtwelve= df.iloc[:, 338].astype(int)
xthirteen= df.iloc[:, 339].astype(str)
xfourteen= df.iloc[:, 340].astype(str)
xfifteen= df.iloc[:, 341].astype(int)
xsixteen= df.iloc[:, 342].astype(str)
xseventeen= df.iloc[:, 343].astype(int)
xeighteen= df.iloc[:, 344].astype(str)
xnineteen= df.iloc[:, 345].astype(int)
xtwenty= df.iloc[:, 346].astype(str)
xtwentyone= df.iloc[:, 347].astype(str)
xtwentytwo= df.iloc[:, 348:675].astype(int)
xztwo= df.iloc[:, 676].astype(str)
xzthree= df.iloc[:, 677].astype(int)
xzfour= df.iloc[:, 678].astype(str)
xzfive= df.iloc[:, 679].astype(int)
xzsix= df.iloc[:, 680].astype(str)
xzseven= df.iloc[:, 681].astype(str)
xzeight= df.iloc[:, 682].astype(int)
xznine= df.iloc[:, 683].astype(str)
xzten= df.iloc[:, 684].astype(int)
xzeleven= df.iloc[:, 685].astype(str)
xztwelve= df.iloc[:, 686].astype(int)
xzthirteen= df.iloc[:, 687].astype(str)
xzfourteen= df.iloc[:, 688].astype(str)
xzfifteen= df.iloc[:, 689].astype(int)
xzsixteen= df.iloc[:, 690].astype(str)
xzseventeen= df.iloc[:, 691].astype(int)
xzeighteen= df.iloc[:, 692].astype(str)
xznineteen= df.iloc[:, 693].astype(int)
xztwenty= df.iloc[:, 694].astype(str)
xztwentyone= df.iloc[:, 695].astype(str)
X_c = [xone,xtwo,xthree,xfour,xfive,xsix,xseven,xeight,xnine,xten,xeleven,xtwelve,xthirteen,xfourteen,xfifteen,xsixteen,xseventeen,xeighteen,xnineteen,xtwenty,xtwentyone,xtwentytwo,xztwo,xzthree,xzfour,xzfive,xzsix,xzseven,xzeight,xznine,xzten,xzeleven,xztwelve,xzthirteen,xzfourteen,xzfifteen,xzsixteen,xzseventeen,xzeighteen,xznineteen,xztwenty,xztwentyone]
X = pd.concat(X_c)
"""

"""
currPrc = 0
for b in range(len(X_train)-1):
    if currPrc == int(b/((len(X_train)-1)*2)*100):
        pass
    else:
        print(str(int(b/((len(X_train)-1)*2)*100))+"%")
        currPrc = int(b/((len(X_train)-1)*2)*100)
    for c in range(695):
        if c+1== 329 or c+1==331 or c+1==333 or c+1==334 or c+1==336 or c+1==338 or c+1==340 or c+1==341 or c+1==343 or c+1==345 or c+1==347 or c+1==348 or c+1==677 or c+1==679 or c+1==681 or c+1==682 or c+1==684 or c+1==686 or c+1==688 or c+1==689 or c+1==691 or c+1==693 or c+1==695 or c+1==696:
            X_train.iloc[b,c]=X_train.iloc[b,c].astype(str)
        else:
            X_train.iloc[b,c]=X_train.iloc[b,c].astype(int)
currPrc = 0
for b in range(len(X_test)-1):
    if currPrc == int(b/((len(X_test)-1)*2)*100+50):
        pass
    else:
        print(str(int(b/((len(X_test)-1)*2)*100+50))+"%")
        currPrc = int(b/((len(X_test)-1)*2)*100+50)
    for c in range(695):
        if c+1== 329 or c+1==331 or c+1==333 or c+1==334 or c+1==336 or c+1==338 or c+1==340 or c+1==341 or c+1==343 or c+1==345 or c+1==347 or c+1==348 or c+1==677 or c+1==679 or c+1==681 or c+1==682 or c+1==684 or c+1==686 or c+1==688 or c+1==689 or c+1==691 or c+1==693 or c+1==695 or c+1==696:
            X_test.iloc[b,c]=X_test.iloc[b,c].astype(str)
        else:
            X_test.iloc[b,c]=X_test.iloc[b,c].astype(int)
    """
"""
for b in range(len(y_test)-1):
    y_test.iloc[b]=y_test.iloc[b].astype(int)
for b in range(len(y_train)-1):
    y_train.iloc[b]=y_train.iloc[b].astype(int)
    """